In [1]:
#Data from https://www.kaggle.com/competitions/open-problems-single-cell-perturbations/data
#Data size is relatively small
#Train with more data for better results.
#Might contain bugs that need to be reviewed.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [51]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 14.9 MB/s eta 0:00:00


In [2]:
cd drive/MyDrive/

/content/drive/MyDrive


In [3]:
import numpy as np
import pandas as pd
import keras
import sys
import tensorflow as tf

In [34]:
df = pd.read_parquet("de_train.parquet")

In [35]:
df.head()

,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,NK cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.104720,-0.077524,-1.625596,-0.144545,0.143555,...,-0.227781,-0.010752,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755
1,T cells CD4+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.915953,-0.884380,0.371834,-0.081677,-0.498266,...,-0.494985,-0.303419,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887
2,T cells CD8+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,-0.387721,-0.305378,0.567777,0.303895,-0.022653,...,-0.119422,-0.033608,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365
3,T regulatory cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.232893,0.129029,0.336897,0.486946,0.767661,...,0.451679,0.704643,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024
4,NK cells,Mometasone Furoate,LSM-3349,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,False,4.290652,-0.063864,-0.017443,-0.541154,0.570982,...,0.758474,0.510762,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629


In [36]:
smiles=df['SMILES']

In [37]:
df = df.drop(['sm_name', 'sm_lincs_id','control', 'SMILES'], axis=1)

In [38]:
df.head()

,cell_type,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,NK cells,0.104720,-0.077524,-1.625596,-0.144545,0.143555,0.073229,-0.016823,0.101717,-0.005153,...,-0.227781,-0.010752,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755
1,T cells CD4+,0.915953,-0.884380,0.371834,-0.081677,-0.498266,0.203559,0.604656,0.498592,-0.317184,...,-0.494985,-0.303419,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887
2,T cells CD8+,-0.387721,-0.305378,0.567777,0.303895,-0.022653,-0.480681,0.467144,-0.293205,-0.005098,...,-0.119422,-0.033608,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365
3,T regulatory cells,0.232893,0.129029,0.336897,0.486946,0.767661,0.718590,-0.162145,0.157206,-3.654218,...,0.451679,0.704643,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024
4,NK cells,4.290652,-0.063864,-0.017443,-0.541154,0.570982,2.022829,0.600011,1.231275,0.236739,...,0.758474,0.510762,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629


In [39]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
import pandas as pd
import numpy as np

class CustomPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.scaler = StandardScaler()
        self.num_imputer = SimpleImputer(strategy='mean')
        self.cat_imputer = SimpleImputer(strategy='most_frequent')

        # Define column names
        self.categorical_column = 'cell_type'
        self.label_encoder = LabelEncoder()
        self.fitted = False

    def fit(self, X, y=None):
        X = X.copy()

        # Separate columns
        self.numerical_columns = [col for col in X.columns if col != self.categorical_column]

        # Fit imputer and scaler on numerical columns
        self.num_imputer.fit(X[self.numerical_columns])
        X_num_imputed = self.num_imputer.transform(X[self.numerical_columns])
        self.scaler.fit(X_num_imputed)

        # Fit imputer and label encoder on cell_type
        self.cat_imputer.fit(X[[self.categorical_column]])
        cell_type_imputed = self.cat_imputer.transform(X[[self.categorical_column]]).ravel()
        self.label_encoder.fit(cell_type_imputed)

        self.fitted = True
        return self

    def transform(self, X):
        if not self.fitted:
            raise RuntimeError("You must fit the preprocessor before calling transform.")

        X = X.copy()

        # Impute and scale numerical columns
        X_num_imputed = self.num_imputer.transform(X[self.numerical_columns])
        X_num_scaled = self.scaler.transform(X_num_imputed)

        # Impute and encode categorical column
        cell_type_imputed = self.cat_imputer.transform(X[[self.categorical_column]]).ravel()
        cell_type_encoded = self.label_encoder.transform(cell_type_imputed).reshape(-1, 1)

        # Combine and return
        X_processed = np.hstack([cell_type_encoded, X_num_scaled])
        return X_processed

In [40]:
#Preprocess
preprocessor = CustomPreprocessor()
X_transformed = preprocessor.fit_transform(df)

In [41]:
print(X_transformed.shape)

(614, 18212)


In [42]:
num_num = len(preprocessor.numerical_columns)
print("Numerical feature columns:", preprocessor.numerical_columns)

Numerical feature columns: ['A1BG', 'A1BG-AS1', 'A2M', 'A2M-AS1', 'A2MP1', 'A4GALT', 'AAAS', 'AACS', 'AAGAB', 'AAK1', 'AAMDC', 'AAMP', 'AAR2', 'AARS', 'AARS2', 'AARSD1', 'AASDH', 'AASDHPPT', 'AASS', 'AATBC', 'AATF', 'AATK', 'ABALON', 'ABAT', 'ABCA1', 'ABCA10', 'ABCA11P', 'ABCA2', 'ABCA3', 'ABCA5', 'ABCA6', 'ABCA7', 'ABCA9', 'ABCB1', 'ABCB10', 'ABCB11', 'ABCB4', 'ABCB7', 'ABCB8', 'ABCB9', 'ABCC1', 'ABCC10', 'ABCC2', 'ABCC3', 'ABCC4', 'ABCC5', 'ABCD1', 'ABCD2', 'ABCD3', 'ABCD4', 'ABCE1', 'ABCF1', 'ABCF2', 'ABCF3', 'ABCG1', 'ABCG2', 'ABCG8', 'ABHD1', 'ABHD10', 'ABHD11', 'ABHD12', 'ABHD13', 'ABHD14A', 'ABHD14A-ACY1', 'ABHD14B', 'ABHD15', 'ABHD15-AS1', 'ABHD16A', 'ABHD17A', 'ABHD17B', 'ABHD17C', 'ABHD18', 'ABHD2', 'ABHD3', 'ABHD4', 'ABHD5', 'ABHD6', 'ABHD8', 'ABI1', 'ABI2', 'ABI3', 'ABL1', 'ABL2', 'ABLIM1', 'ABLIM2', 'ABO', 'ABR', 'ABRACL', 'ABRAXAS1', 'ABRAXAS2', 'ABT1', 'ABTB1', 'ABTB2', 'AC000068.1', 'AC000123.2', 'AC000123.3', 'AC000403.1', 'AC001226.2', 'AC002044.1', 'AC002044.2', 'AC0

In [43]:
cat_dim = len(preprocessor.label_encoder.classes_)
print("Categorical feature and label-encoded class count:")
print(f"  cell_type: {cat_dim} unique labels")

print("Total features after transform:", 1 + len(preprocessor.numerical_columns))

Categorical feature and label-encoded class count:
  cell_type: 6 unique labels
Total features after transform: 18212


In [44]:
cat_dims = [len(preprocessor.label_encoder.classes_)]  # only 'cell_type'
num_num = len(preprocessor.numerical_columns)          # numeric gene columns

print("Categorical feature and label-encoded class count:")
print(f"  cell_type: {cat_dims[0]} unique labels")

print("Total numerical features:", num_num)
print("Total features after transform:", 1 + num_num)  # 1 for 'cell_type'

Categorical feature and label-encoded class count:
  cell_type: 6 unique labels
Total numerical features: 18211
Total features after transform: 18212


In [45]:
smiles=np.array(smiles)

In [48]:
print(smiles.shape)

(614,)


In [50]:
print(X_transformed.shape)

(614, 18212)


In [56]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def tokenize_smiles(smiles_list):
    # 1. Extract unique characters from SMILES strings
    charset = sorted(set("".join(smiles_list)))

    # 2. Create mapping from char → index (1-based indexing; 0 = padding)
    char_to_idx = {char: i + 1 for i, char in enumerate(charset)}
    idx_to_char = {i + 1: char for i, char in enumerate(charset)}

    # 3. Tokenize each SMILES into a list of indices
    encoded = [[char_to_idx[char] for char in smile] for smile in smiles_list]

    # 4. Pad sequences to the same length
    max_length = max(len(seq) for seq in encoded)
    padded = pad_sequences(encoded, maxlen=max_length, padding='post', value=0)

    return np.array(padded), char_to_idx, idx_to_char, max_length

# === Tokenize your SMILES strings ===
smiles_encoded, char_to_idx, idx_to_char, max_smiles_length = tokenize_smiles(smiles)

In [139]:
#Wasserstein Autoencoder
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, regularizers, Model, Input
from tensorflow.keras.models import Model

def build_encoder(input_dim, latent_dim, timesteps=None, noise_stddev=0.1, dropout_rate=0.2, l2_reg=1e-4):
    if timesteps is not None:
        inputs = Input(shape=(timesteps, input_dim))  # Sequence input
        x = layers.Masking(mask_value=0.0)(inputs)
        x = layers.LSTM(64, return_sequences=True, kernel_regularizer=regularizers.l2(l2_reg))(x)
        x = layers.Dropout(dropout_rate)(x)
        x = layers.LSTM(32, return_sequences=False, kernel_regularizer=regularizers.l2(l2_reg))(x)
    else:
        inputs = Input(shape=(input_dim,))  # Flat input
        x = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(l2_reg))(inputs)
        x = layers.Dropout(dropout_rate)(x)
        x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(l2_reg))(x)

    x = layers.GaussianNoise(noise_stddev)(x)
    z = layers.Dense(latent_dim, name='latent_vector')(x)
    return Model(inputs, z, name="encoder")

def build_decoder(latent_dim, vocab_size, max_length, embed_dim=64, l2_reg=1e-4):
    latent_inputs = Input(shape=(latent_dim,), name='decoder_input')

    # Project latent vector to an embedding sequence
    x = layers.Dense(max_length * embed_dim, activation='relu',
                     kernel_regularizer=regularizers.l2(l2_reg))(latent_inputs)
    x = layers.Reshape((max_length, embed_dim))(x)

    # RNN to generate token sequence
    x = layers.GRU(128, return_sequences=True,
                   kernel_regularizer=regularizers.l2(l2_reg))(x)
    outputs = layers.TimeDistributed(layers.Dense(vocab_size + 1, activation='softmax'))(x)  # +1 for padding

    return Model(latent_inputs, outputs, name="decoder")

def build_critic(latent_dim, l2_reg=1e-4):
    inputs = Input(shape=(latent_dim,), name='critic_input')
    x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(l2_reg))(inputs)
    x = layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(l2_reg))(x)
    out = layers.Dense(1)(x)  # Wasserstein scalar score
    return Model(inputs, out, name="critic")

In [140]:
import numpy as np
import tensorflow as tf
from tqdm import tqdm

# === Assume these are already prepared ===
# X_transformed: shape (N, num_features)
# smiles_encoded: shape (N, max_length), tokenized SMILES strings
# char_to_idx, idx_to_char: tokenizer mappings

# === Params ===
latent_dim = 10
batch_size = 256
epochs = 1500
critic_iters = 3
lambda_gp = 10.0
vocab_size = len(char_to_idx)
max_smiles_length = smiles_encoded.shape[1]
input_dim = X_transformed.shape[1]

# === Build models ===
encoder = build_encoder(input_dim, latent_dim)
decoder = build_decoder(latent_dim, vocab_size, max_smiles_length)
critic = build_critic(latent_dim)

# === Optimizers ===
optimizer_gen = tf.keras.optimizers.Adam(1e-4, beta_1=0.5, beta_2=0.9)
optimizer_critic = tf.keras.optimizers.Adam(1e-4, beta_1=0.5, beta_2=0.9)

# === Dataset ===
dataset = tf.data.Dataset.from_tensor_slices((X_transformed.astype(np.float32), smiles_encoded.astype(np.int32)))
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size).cache().prefetch(tf.data.AUTOTUNE)

# === Warm-up ===
_ = encoder(X_transformed[:1])
_ = decoder(tf.random.normal([1, latent_dim]))
_ = critic(tf.random.normal([1, latent_dim]))

@tf.function
def train_critic(batch_x):
    critic_loss = 0.0
    for _ in tf.range(critic_iters):
        with tf.GradientTape() as tape:
            z = encoder(batch_x)
            prior_z = tf.random.normal(shape=tf.shape(z), dtype=tf.float32)

            critic_z = critic(z)
            critic_prior = critic(prior_z)

            alpha = tf.random.uniform(shape=[tf.shape(z)[0], 1], dtype=tf.float32)
            interpolates = alpha * z + (1.0 - alpha) * prior_z

            with tf.GradientTape() as gp_tape:
                gp_tape.watch(interpolates)
                critic_interpolates = critic(interpolates)

            gradients = gp_tape.gradient(critic_interpolates, [interpolates])[0]
            slopes = tf.sqrt(tf.reduce_sum(tf.square(gradients), axis=1) + 1e-12)
            gradient_penalty = tf.reduce_mean(tf.square(slopes - 1.0))

            loss = tf.reduce_mean(critic_prior) - tf.reduce_mean(critic_z)
            loss += lambda_gp * gradient_penalty

        grads = tape.gradient(loss, critic.trainable_variables)
        optimizer_critic.apply_gradients(zip(grads, critic.trainable_variables))
        critic_loss += loss
    return critic_loss / tf.cast(critic_iters, tf.float32)

@tf.function
def train_generator(batch_x, batch_smiles):
    with tf.GradientTape() as tape:
        z = encoder(batch_x)
        decoded = decoder(z)  # shape: (batch_size, max_length, vocab_size + 1)
        critic_z = critic(z)

        # Reconstruction loss: categorical crossentropy per token
        recon_loss = tf.reduce_mean(
            tf.keras.losses.sparse_categorical_crossentropy(batch_smiles, decoded)
        )

        gen_loss = recon_loss + tf.reduce_mean(critic_z)

    grads = tape.gradient(gen_loss, encoder.trainable_variables + decoder.trainable_variables)
    optimizer_gen.apply_gradients(zip(grads, encoder.trainable_variables + decoder.trainable_variables))
    return gen_loss

# === Training Loop ===
num_batches = tf.data.experimental.cardinality(dataset).numpy()

for epoch in range(epochs):
    epoch_gen_loss = []
    epoch_critic_loss = []

    print(f"\n=== Epoch {epoch+1}/{epochs} ===", flush=True)
    for batch_idx, (batch_x, batch_smiles) in enumerate(tqdm(dataset, total=num_batches, desc=f"Epoch {epoch+1}")):
        batch_x = tf.cast(batch_x, tf.float32)
        batch_smiles = tf.cast(batch_smiles, tf.int32)

        critic_loss = train_critic(batch_x)
        gen_loss = train_generator(batch_x, batch_smiles)

        epoch_critic_loss.append(critic_loss.numpy())
        epoch_gen_loss.append(gen_loss.numpy())

        if (batch_idx + 1) % 50 == 0 or (batch_idx + 1) == num_batches:
            print(f"Batch {batch_idx + 1}/{num_batches}: Gen Loss = {gen_loss.numpy():.4f}, Critic Loss = {critic_loss.numpy():.4f}", flush=True)

    print(f"\n✔ Epoch {epoch+1} Complete | Avg Gen Loss: {np.mean(epoch_gen_loss):.6f}, Avg Critic Loss: {np.mean(epoch_critic_loss):.6f}", flush=True)


=== Epoch 1/1500 ===


Epoch 1:  33%|███▎      | 1/3 [00:07<00:15,  7.91s/it]

Batch 3/3: Gen Loss = 2.8957, Critic Loss = 4.5625


Epoch 1: 100%|██████████| 3/3 [00:11<00:00,  4.00s/it]


✔ Epoch 1 Complete | Avg Gen Loss: 3.228852, Avg Critic Loss: 4.311481

=== Epoch 2/1500 ===


...


✔ Epoch 1499 Complete | Avg Gen Loss: 2.086360, Avg Critic Loss: 0.541029

=== Epoch 1500/1500 ===



Epoch 1500:  67%|██████▋   | 2/3 [00:00<00:00, 11.79it/s]

Batch 3/3: Gen Loss = 2.3868, Critic Loss = 0.2548


Epoch 1500: 100%|██████████| 3/3 [00:00<00:00,  9.91it/s]


✔ Epoch 1500 Complete | Avg Gen Loss: 2.098119, Avg Critic Loss: 0.502582


In [143]:
import numpy as np

def generate_smiles(input_vector, encoder, decoder, idx_to_char, preprocessor=None, num_samples=1, max_length=None):
    """
    Generate SMILES strings from a single input vector, optionally preprocessing it first.

    Args:
        input_vector (np.ndarray or pd.Series or dict): raw or preprocessed input vector
        encoder (tf.keras.Model): trained encoder model
        decoder (tf.keras.Model): trained decoder model
        idx_to_char (dict): mapping from token index to character
        preprocessor (CustomPreprocessor, optional): preprocess input if provided
        num_samples (int): number of SMILES strings to generate
        max_length (int, optional): max length of output sequences; inferred if None

    Returns:
        list of str: list of generated SMILES strings
    """
    # Preprocess if needed
    if preprocessor is not None:
        import pandas as pd
        if isinstance(input_vector, dict):
            input_vector = pd.DataFrame([input_vector])
        elif isinstance(input_vector, np.ndarray) and input_vector.ndim == 1:
            input_vector = pd.DataFrame([input_vector])
        elif isinstance(input_vector, pd.Series):
            input_vector = input_vector.to_frame().T
        input_vector = preprocessor.transform(input_vector)

    # Prepare input for encoder
    input_vector = np.array(input_vector, dtype=np.float32)
    if input_vector.ndim == 1:
        input_vector = np.expand_dims(input_vector, axis=0)

    # Encode to latent space
    z = encoder.predict(input_vector, verbose=0)

    # Decode sequence probabilities
    decoded_seq = decoder.predict(z, verbose=0)  # shape: (1, max_length, vocab_size+1)

    if max_length is None:
        max_length = decoded_seq.shape[1]

    vocab_size_plus = decoded_seq.shape[2]

    samples = []
    for _ in range(num_samples):
        smiles_chars = []
        for t in range(max_length):
            # Get probability distribution for timestep t
            probs = decoded_seq[0, t]
            # Sample token index from probs
            token_idx = np.random.choice(vocab_size_plus, p=probs/probs.sum())
            if token_idx == 0:  # padding or stop token, stop generation
                break
            smiles_chars.append(idx_to_char.get(token_idx, ''))
        smiles = ''.join(smiles_chars)
        samples.append(smiles)

    return samples if num_samples > 1 else samples[0]

In [155]:
# Generate molecules for some input. Add the preprocessor if entering raw input.
generated_smiles_list = generate_smiles(X_transformed[0], encoder, decoder, idx_to_char, num_samples=30000)

In [156]:
from rdkit import Chem

def filter_valid_smiles(smiles_list):
    valid_smiles = []
    for smi in smiles_list:
        mol = Chem.MolFromSmiles(smi)
        if mol is not None and len(smi) >= 5:
            valid_smiles.append(smi)
    return valid_smiles

In [157]:
valid_smiles = filter_valid_smiles(generated_smiles_list)

In [158]:
len(valid_smiles)

4

In [159]:
valid_smiles  #Train with more examples for better results

['CCCC1cN1', 'CCC1cC1', 'CCNCC', 'O1ONC1']

In [ ]:
#Optional plotting for diversity

# from rdkit import Chem, DataStructs
# from rdkit.Chem import AllChem
# import matplotlib.pyplot as plt
# from sklearn.manifold import TSNE
# import numpy as np

# def plot_smiles_diversity(smiles_list):
#     # Convert SMILES to Morgan fingerprints
#     fps = []
#     for smi in smiles_list:
#         mol = Chem.MolFromSmiles(smi)
#         if mol is not None:
#             fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=1024)
#             fps.append(fp)
#
#     # Convert fingerprints to numpy array
#     arr = []
#     for fp in fps:
#         arr.append(np.array(list(fp.ToBitString()), dtype=int))
#     arr = np.array(arr)
#
#     # Use t-SNE for 2D visualization
#     tsne = TSNE(n_components=2, random_state=42)
#     reduced = tsne.fit_transform(arr)
#
#     # Plot the diversity
#     plt.figure(figsize=(8, 6))
#     plt.scatter(reduced[:, 0], reduced[:, 1], alpha=0.7)
#     plt.title('Diversity Plot of Valid SMILES')
#     plt.xlabel('t-SNE Dimension 1')
#     plt.ylabel('t-SNE Dimension 2')
#     plt.show()